In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors


In [24]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [25]:
encoder = LabelEncoder()
final_vectors = user_vectors.copy()
x = encoder.fit(final_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
username_enc

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc

#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#encoder.inverse_transform([7537])

Compute K-Nearest Neighbours

In [32]:
#function that returns a list of indexes of most similar users in "final_arrays" array
def neighbours():
    user = np.array([7537,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data
    n = train.kneighbors([user], return_distance=False)
    return n

sim_names = []
neighbours = neighbours()
for i in neighbours[0]: #loop through nested array
    #print(final_arrays[i][0])
    name = x.inverse_transform([final_arrays[i][0]])
    sim_names.append(name)

#build new dataframe of neighbours
neighbour_frame = pd.DataFrame()
for i in users.columns:
    neighbour_frame[i] = None

for name in sim_names:
    user_row = users.loc[users["username"]==name[0]]
    neighbour_frame = pd.concat([neighbour_frame, user_row])
neighbour_frame


#(final_vectors.columns)    

,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
0,2255153,karthiga,49,Active,Female,1990,3391,Millenials,"['Comedy', 'Action', 'Shounen']",Classic
68,502880,lila94,89,Active,Female,1994,4204,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
416,2120443,hana_haruka,110,Active,Female,1993,4062,Millenials,"['Action', 'Shounen', 'Comedy']",Classic
3962,71147,gosakurago,86,Active,Female,1994,3329,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
3132,487038,gorankudasai,144,Active,Female,1992,3513,Millenials,"['Action', 'Shounen', 'Comedy']",Classic


Create anime recommendations

In [36]:

#Put anime that have most members at the top of each users "lists"
def recommendations(neighbours):
    #Get top rated show from each user
    suggestions = [] #anime_id's of recommended anime
    for i in neighbours["username"]:
        user_ratings = ratings.loc[ratings["username"]==i]
        top_rated = user_ratings.iloc[0]["title"]
        suggestions.append(top_rated)
        #print(top_rated)
    return suggestions

recommendations(neighbour_frame)
#ratings

['Kamisama Hajimemashita',
 'Katekyo Hitman Reborn!',
 'Gintama&#039;: Enchousen',
 'Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.',
 'Mononoke Hime']

Function that converts user information from website into vector